### EDA and Data Cleaning of Web-Scraped Reddit Posts

In [1]:
import requests
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.max_columns', None)

In [2]:
sci = pd.read_csv('../datasets/science.csv')
phi = pd.read_csv('../datasets/philosophy.csv')

C:\Users\barar\AppData\Local\Temp\ipykernel_31204\3484891841.py:1: DtypeWarning: Columns (71,72,78,81) have mixed types. Specify dtype option on import or set low_memory=False.
  sci = pd.read_csv('../datasets/science.csv')
C:\Users\barar\AppData\Local\Temp\ipykernel_31204\3484891841.py:2: DtypeWarning: Columns (3,8,19,75,76,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  phi = pd.read_csv('../datasets/philosophy.csv')


#### Science subreddit

In [9]:
sci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23704 entries, 0 to 25085
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   all_awardings                  23704 non-null  object 
 1   allow_live_comments            23704 non-null  bool   
 2   author                         23704 non-null  object 
 3   author_flair_css_class         1536 non-null   object 
 4   author_flair_richtext          23504 non-null  object 
 5   author_flair_text              497 non-null    object 
 6   author_flair_type              23504 non-null  object 
 7   author_fullname                23504 non-null  object 
 8   author_is_blocked              23704 non-null  bool   
 9   author_patreon_flair           23504 non-null  object 
 10  author_premium                 23504 non-null  object 
 11  awarders                       23704 non-null  object 
 12  can_mod_post                   23704 non-null 

In [4]:
sci.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_created_from_ads_ui,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_template_id,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,total_awards_received,treatment_tags,upvote_ratio,url,url_overridden_by_dest,whitelist_status,wls,removed_by_category,author_flair_background_color,author_flair_text_color,collections,distinguished,author_flair_template_id,media,media_embed,secure_media,secure_media_embed,author_cakeday,call_to_action,category,media_metadata,gilded
0,[],False,chilladipa,NaN,[],NaN,text,t2_6z8harzu,False,False,False,[],False,False,1664811669,nobelprize.org,https://www.reddit.com/r/science/comments/xumv...,{},xumvdn,False,True,False,False,False,True,False,False,#00a6a5,gen,[],54449530-7df9-11ec-b93b-a6179baa4180,Genetics,light,text,False,False,True,1,0,False,all_ads,/r/science/comments/xumvdn/2022_nobel_prize_in...,False,link,"{'enabled': False, 'images': [{'id': '5QckU-T3...",6,1664811680,1,NaN,True,False,False,science,t5_mouw,28362053,public,confidence,https://b.thumbs.redditmedia.com/SmjImtboLgWPN...,73.0,140.0,2022 Nobel Prize in Physiology or Medicine to ...,0,[],1.0,https://www.nobelprize.org/prizes/medicine/202...,https://www.nobelprize.org/prizes/medicine/202...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Simple-Rest-8625,NaN,[],NaN,text,t2_stwpl0q0,False,False,False,[],False,False,1664809032,playeffort.com,https://www.reddit.com/r/science/comments/xulp...,{},xulpxy,False,False,False,False,False,False,False,False,#d08dd2,physics,[],084a5ff8-6806-11e5-9958-12c8a3890827,Physics,light,text,False,False,True,1,0,False,all_ads,/r/science/comments/xulpxy/india_vs_south_afri...,False,link,"{'enabled': False, 'images': [{'id': 'xh2fDbeQ...",6,1664809042,1,NaN,True,False,False,science,t5_mouw,28361753,public,confidence,https://b.thumbs.redditmedia.com/l8Wt7xMrhqyrh...,93.0,140.0,India vs South Africa 2nd T20,0,[],1.0,https://www.playeffort.com/2022/10/india-vs-so...,https://www.playeffort.com/2022/10/india-vs-so...,all_ads,6,reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,patient-Spring-4,NaN,[],NaN,text,t2_7qho2c7l,False,False,False,[],False,False,1664807875,inverse.com,https://www.reddit.com/r/science/comments/xul8...,{},xul8ap,False,False,False,False,False,False,False,False,#7ac877,animalsci,[],109e172c-4638-11e3-abe9-12313b04ceaf,Animal Science,light,text,False,False,True,1,0,False,all_ads,/r/science/comments/xul8ap/want_a_better_relat...,False,link,"{'enabled': False, 'images': [{'id': '5M0zlHk0...",6,1664807886,1,NaN,False,False,False,science,t5_mouw,28361628,public,confidence,https://b.thumbs.redditmedia.com/0nuJ5dIdvsTVM...,73.0,140.0,Want a better relationship with your dog? Thes...,0,[],1.0,https://www.inverse.com/innovation/seven-facto...,https://www.inverse.com/innovation/seven-facto...,all_ads,6,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,trevor25,NaN,[],NaN,text,t2_jeng2,False,False,False,[],False,False,1664807763,bbc.com,https://www.reddit.com/r/science/comments/xul6...,{},xul6ka,False,True,False,False,False,True,False,False,#7ac877,animalsci,[],109e172c-4638-11e3-abe9-12313b04ceaf,Animal Science,light,text,False,False,True,1,0,False,all_ads,/r/science/comments/xul6ka/record_avian_flu_ou...,False,link,"

In [5]:
# Remove duplicate posts
sci.drop_duplicates(subset=['title'], inplace = True)

In [6]:
# create a subset df for our subsequent analysis
sci_df = sci[['subreddit', 'title', 'author', 'link_flair_text']].copy()

In [7]:
sci_df.rename(columns = {"title": "posts", "link_flair_text": "category"}, inplace = True)

In [8]:
sci_df.head()

,subreddit,posts,author,category
0,science,2022 Nobel Prize in Physiology or Medicine to ...,chilladipa,Genetics
1,science,India vs South Africa 2nd T20,Simple-Rest-8625,Physics
2,science,Want a better relationship with your dog? Thes...,patient-Spring-4,Animal Science
3,science,Record avian flu outbreak sees 48m birds culle...,trevor25,Animal Science
4,science,"In this review article, we focus on the effect...",Meatrition,Health


#### Philosophy subreddit

In [10]:
phi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25113 entries, 0 to 25112
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   all_awardings                  25113 non-null  object 
 1   allow_live_comments            25113 non-null  bool   
 2   author                         25113 non-null  object 
 3   author_flair_css_class         1 non-null      object 
 4   author_flair_richtext          24941 non-null  object 
 5   author_flair_text              496 non-null    object 
 6   author_flair_type              24941 non-null  object 
 7   author_fullname                24941 non-null  object 
 8   author_is_blocked              22307 non-null  object 
 9   author_patreon_flair           24941 non-null  object 
 10  author_premium                 24941 non-null  object 
 11  awarders                       25113 non-null  object 
 12  can_mod_post                   25113 non-null 